# Exemple de réseaux récurent prédisant du texte
## Utilisation de l'architecture GRU

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from itertools import islice


In [3]:
# Quelques phrases simples
sentences = [
    "la goutte d'eau qui fait déborder le vase", 
    "Il n'y a pas de fumée sans feu", 
    "Il faut battre le fer tant qu'il est chaud", 
    "Il ne faut pas mettre tous ses oeufs dans le même panier", 
    "Il faut tourner sept fois sa langue dans sa bouche avant de parler", 
    "L'habit ne fait pas le moine", 
    "Il ne faut pas réveiller le chat qui dort", 
    "Il faut se méfier de l'eau qui dort", 
    "C'est l'hôpital qui se moque de la charité", 
    "Qui vole un oeuf vole un boeuf", 
    "Chercher midi à quatorze heures", 
    "Avoir un poil dans la main", 
    "Être dans de beaux draps", 
    "Avoir la tête dans les nuages", 
    "Mettre les pieds dans le plat"]

---
### Création du vocabulaire 
Premièrement, on analyse tous les textes pour récupérer l'ensembles des mots utilisés et on numérote les tokens (mots), chaque mot a ainsi un indice unique.

Un objet Tokenizer sera utile.

In [ ]:
# Récupération des tokens : le Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
#tokenizer.word_index est un dictionnaire (une map), on le transforme en liste de mots (liste[0] = mot 1 (0+1))  
liste = list(tokenizer.word_index.keys())

print("nb de mots différents rencontrés :", total_words)
for key, value in islice(tokenizer.word_index.items(), 10):
    print(f"{key}: {value}", end=", ")
print()

nb de mots différents rencontrés : 72
le: 1, il: 2, dans: 3, qui: 4, de: 5, faut: 6, la: 7, pas: 8, ne: 9, un: 10, 


---
### Transformer un texte en vecteur
Maintenant, on remplace chaque mot, token, par son indice pour créer 1 vecteur d'entiers à partir d'une chaîne de caractères.

In [29]:
phrase0 = sentences[0]
vecteur0 = tokenizer.texts_to_sequences([phrase0])[0]
print(phrase0)
print("est traduit en")
print(vecteur0)

la goutte d'eau qui fait déborder le vase
est traduit en
[7, 19, 20, 4, 11, 21, 1, 22]


Maintenant, on souhaite apprendre progressivement la suite d'une phrase : 
- la goutte 
- la goutte d'eau 
- la goutte d'eau qui 
- la goutte d'eau qui fait
- la goutte d'eau qui fait déborder 
- la goutte d'eau qui fait déborder le 
- la goutte d'eau qui fait déborder le vase

In [ ]:
# transformation des textes en vecteurs 
input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

In [31]:
input_sequences[:5]

[[7, 19],
 [7, 19, 20],
 [7, 19, 20, 4],
 [7, 19, 20, 4, 11],
 [7, 19, 20, 4, 11, 21]]

In [48]:
# calibrage des vecteurs pour qu'ils aient tous la même longueur
max_sequence_len = max([len(x) for x in input_sequences])
# on rempli les cases "pre"cedentes de 0 si la longueur du vecteur < max_sequence_len
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [49]:
print("la phrase '", sentences[0], "' est traduite en plusieurs vecteurs de même taille :")
split = sentences[0].split()
for i in range(6):
    print(input_sequences[i], end=" -> '")
    for j in range(i+2):
        print(split[j], end=" ")
    print("'")

la phrase ' la goutte d'eau qui fait déborder le vase ' est traduite en plusieurs vecteurs de même taille :
[ 0  0  0  0  0  0  0  0  0  0  0  7 19] -> 'la goutte '
[ 0  0  0  0  0  0  0  0  0  0  7 19 20] -> 'la goutte d'eau '
[ 0  0  0  0  0  0  0  0  0  7 19 20  4] -> 'la goutte d'eau qui '
[ 0  0  0  0  0  0  0  0  7 19 20  4 11] -> 'la goutte d'eau qui fait '
[ 0  0  0  0  0  0  0  7 19 20  4 11 21] -> 'la goutte d'eau qui fait déborder '
[ 0  0  0  0  0  0  7 19 20  4 11 21  1] -> 'la goutte d'eau qui fait déborder le '


In [50]:
# creer les x (les  valeurs de chaque vecteur sauf la dernière)
X = input_sequences[:, :-1]
# creer les y (la derniere valeur de chaque vecteur)
y = input_sequences[:, -1]

In [51]:
# chaque mot de sortie est représenté par un vecteur de 0s,  seul l'indice du mot = 1 
#donc le vecteur est aussi grand que le nb de mots trouvés
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [60]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

---
### Le modèle de réseau
Spécialement pour le texte, on décide de représenter un mot, non plus par un entier, mais par un vecteur de caractéristiques qui le représente.

La couche **Embedding** permet cette transformation. 
- Théoriquement, si chat=2 et chien=5, un embedding de taille 3 donnera chat=[0.1, -0.4, 0.3] et chien=[0.5, -0.2, 0.5]
- Ces vecteurs s'affine en fonction de l'apprentissage du texte.. A priori, si chien et chat sont utilisés de manière identique 'le chien mange", "le chat mange", ..... au bout d'un certain temps, les vecteurs chat et chien vont posséder des valeurs semblables.

La couche **GRU** est chargée de l'apprentissage des séries de valeurs. On peut utiliser plusieurs couches GRU, return_sequences sera vrai pour toutes sauf la dernière. La taille de la couche est d'environ 100 pour un vocabulaire moyen (qq milliers de mots)

In [74]:
#creation du model
model = Sequential()
## Embedding : chaque mot est représenté par un vecteur de 50 valeurs
model.add(Embedding(total_words, 50))
model.add(GRU(100, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

In [75]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [76]:

# entrainer le modele
print("patienter 30s pendant l'entrainement...")
model.fit(X, y, epochs=300, verbose=0)

patienter 30s pendant l'entrainement...


---
### Prédictions
Pour compléter une phrase, on demande la génération d'un mot, puis la génération d'un nouveau mot complétant la phrase à laquelle on a ajouté le mot précédent, etc. jusqu'au nombre de mots souhaités.  

In [71]:
# fonction pour prédire le mot suivant
def predict_next_word(start_text, next_words=1):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([start_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                start_text += " " + word
                break
    
    return start_text

In [77]:
# Tester le modèle avec quelques textes
start_texts = [
    "il faut tourner sept fois sa  ",
    "la goutte d'eau qui fait  ",
    "Qui vole un oeuf   ",
]

for text in start_texts:
    print(f"first part : {text}")
    nb = 5 if text.count(" ")<5 else 3
    print(f"prédictions de {nb} mots.")
    print(f"Prediction: {predict_next_word(text, nb)}")
    print("-" * 50)

first part : il faut tourner sept fois sa  
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: il faut tourner sept fois sa   langue dans sa
--------------------------------------------------
first part : la goutte d'eau qui fait  
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: la goutte d'eau qui fait   déborder le vase
--------------------------------------------------
first part : Qui vole un oeuf   
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: Qui vole un oeuf    vole un boeuf
--------------------------------------------------


In [78]:
# Tester le modèle avec quelques textes
start_texts = [
    "chercher un poil   ",
    "il faut battre le moine   ",
]

for text in start_texts:
    print(f"first part : {text}")
    nb = 4 #if text.count(" ")<5 else 3
    print(f"prédictions de {nb} mots.")
    print(f"Prediction: {predict_next_word(text, nb)}")
    print("-" * 50)

first part : chercher un poil   
prédictions de 4 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: chercher un poil    dans la main vole
--------------------------------------------------
first part : il faut battre le moine   
prédictions de 4 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: il faut battre le moine    tant qu'il est chaud
--------------------------------------------------


---
### Post traitement
On s'aperçoit de petit bugs, dit "halunications". 
Il faut souvent vérifier les sorties et les corriger (par exemple, empêcher la répétition du même mot, ...)